![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/48dd9de1-b4d2-4318-8f52-85ec209d8ebc)

# Install Necessary Libraries

In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

# Clone a GitHub Repo locally

In [4]:
github_repo = "https://github.com/evanng07/CodeInsight"
github_repo.split("/")[-1]

'CodeInsight'

In [5]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """
    repo_name = github_repo.split("/")[-1]
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return repo_path


In [6]:
path = clone_repository(github_repo)

# Define which types of files to parse and which files / folders to ignore

In [10]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [8]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
      with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

      rel_path = os.path.relpath(file_path, repo_path)

      return {
          "name": rel_path,
          "content": content
      }

    except Exception as e:
      print(f"Error processing file {file_path}: {str(e)}")
      return None



def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """

    files_content = []

    try:

      for root, _, files in os.walk(repo_path):
        if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
          continue

        for file in files:
          file_path = os.path.join(root, file)
          if os.path.splitext(file) [1] in SUPPORTED_EXTENSIONS:
            file_content = get_file_content(file_path, repo_path)

            if file_content:
              files_content.append(file_content)

    except Exception as e:
      print(e)

    return files_content


In [11]:
files_content = get_main_files_content(path)

In [12]:
files_content

[{'name': 'profile_1.py',
  'content': 'import ast\nimport os\nimport json\n\nclass CallGraphAnalyzer(ast.NodeVisitor):\n    def __init__(self, source):\n        self.call_graph = {}\n        self.function_code = {}\n        self.source = source\n        self.current_function = None\n\n    def visit_FunctionDef(self, node):\n        prev_function = self.current_function\n        self.current_function = node.name\n\n        # Initialize call list if not already present.\n        if node.name not in self.call_graph:\n            self.call_graph[node.name] = []\n\n        # Use ast.get_source_segment (available in Python 3.8+)\n        code_snippet = ast.get_source_segment(self.source, node)\n        self.function_code[node.name] = code_snippet if code_snippet else ""\n\n        self.generic_visit(node)\n        self.current_function = prev_function\n\n    def visit_Call(self, node):\n        if isinstance(node.func, ast.Name):\n            func_name = node.func.id\n        elif isinstanc

# Embeddings

In [13]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [14]:
text = "I am learning"
embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
embeddings

array([ 5.30410446e-02, -7.29994252e-02, -5.68303559e-03, -1.70101058e-02,
        1.09569943e-02,  2.93799601e-02, -6.21463880e-02, -5.72640309e-03,
        1.04285469e-02,  3.50432768e-02,  4.13903408e-02, -5.97487278e-02,
        1.26606142e-02, -7.40056019e-03,  3.71821374e-02, -3.92079316e-02,
        8.63618478e-02, -4.59995717e-02,  5.94833458e-04, -1.32530611e-02,
        7.03032687e-03,  1.97139643e-02, -6.11965060e-02, -2.31416952e-02,
       -2.81213727e-02, -4.64432165e-02,  8.90711846e-04, -3.26461233e-02,
       -4.89002168e-02,  1.37800211e-03,  9.60131828e-03, -5.42815775e-02,
       -5.30457310e-03,  7.49727935e-02,  1.92705625e-06, -3.64193805e-02,
       -2.79862694e-02, -2.77878065e-02, -6.22987226e-02,  4.19626720e-02,
        1.33600589e-02,  3.17652114e-02, -9.18876845e-03,  2.14989595e-02,
       -2.50022151e-02,  2.67981496e-02,  7.85335228e-02,  2.16396414e-02,
        1.07983444e-02,  2.68672705e-02, -1.55073043e-03, -5.24942428e-02,
       -2.04970427e-02, -

In [16]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [17]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-17-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-17-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [23]:
# Insert the codebase embeddings into Pinecone

documents = []

for file in files_content:
  doc = Document(
      page_content=f"{file['name']}\n\n{file['content']}",
      metadata={"source": file['name']}
  )

  documents.append(doc)


vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/evanng07/CodeInsight"
)

<ipython-input-23-fe157a2f263d>:16: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


In [20]:
documents

[Document(metadata={'source': 'profile_1.py'}, page_content='profile_1.py\n\nimport ast\nimport os\nimport json\n\nclass CallGraphAnalyzer(ast.NodeVisitor):\n    def __init__(self, source):\n        self.call_graph = {}\n        self.function_code = {}\n        self.source = source\n        self.current_function = None\n\n    def visit_FunctionDef(self, node):\n        prev_function = self.current_function\n        self.current_function = node.name\n\n        # Initialize call list if not already present.\n        if node.name not in self.call_graph:\n            self.call_graph[node.name] = []\n\n        # Use ast.get_source_segment (available in Python 3.8+)\n        code_snippet = ast.get_source_segment(self.source, node)\n        self.function_code[node.name] = code_snippet if code_snippet else ""\n\n        self.generic_visit(node)\n        self.current_function = prev_function\n\n    def visit_Call(self, node):\n        if isinstance(node.func, ast.Name):\n            func_name =

In [50]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY")
)

In [51]:
print(os.getenv("OPENROUTER_API_KEY"))  # Should not be None or empty

None


In [25]:
query = "What is happening in this codebase?"

In [27]:
raw_query_embedding = get_huggingface_embeddings(query)

In [28]:
raw_query_embedding

array([ 6.59794509e-02, -9.47666913e-03, -2.86735576e-02,  7.44199157e-02,
        2.20942107e-04,  5.63381165e-02, -2.64694057e-02,  1.65925303e-03,
       -1.62661634e-02, -2.94047389e-02,  3.34773166e-03,  3.82754914e-02,
        6.03461452e-02,  3.07034254e-02,  5.08948928e-03,  1.38098095e-02,
        3.46340761e-02, -2.41783261e-02, -7.08794966e-03, -2.08355649e-03,
       -1.81135777e-02,  1.46383466e-02, -2.39711627e-03,  3.53236087e-02,
       -1.90799974e-03, -1.75620317e-02,  7.78759131e-03,  7.12758824e-02,
       -4.31852192e-02,  1.16784207e-03, -2.32307743e-02,  7.92281702e-03,
       -3.04139908e-02, -5.68122156e-02,  1.62807908e-06, -4.60486021e-03,
        1.95783470e-02,  1.21030314e-02, -2.27611177e-02,  2.36542579e-02,
       -4.76128014e-04,  5.21754697e-02,  1.29191307e-02,  2.66384762e-02,
       -2.55688615e-02, -5.14759012e-02, -5.88790700e-02, -3.72281745e-02,
       -1.17227603e-02,  5.74757457e-02,  8.36855639e-03, -1.57110672e-03,
       -2.78145093e-02,  

In [33]:
top_mathces = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/evanng07/CodeInsight")

In [34]:
top_mathces

{'matches': [{'id': 'cedd0174-f127-4d38-9f81-fc92cefb2567',
              'metadata': {'source': 'profile_1.py',
                           'text': 'profile_1.py\n'
                                   '\n'
                                   'import ast\n'
                                   'import os\n'
                                   'import json\n'
                                   '\n'
                                   'class CallGraphAnalyzer(ast.NodeVisitor):\n'
                                   '    def __init__(self, source):\n'
                                   '        self.call_graph = {}\n'
                                   '        self.function_code = {}\n'
                                   '        self.source = source\n'
                                   '        self.current_function = None\n'
                                   '\n'
                                   '    def visit_FunctionDef(self, node):\n'
                                   '        prev_fun

In [38]:
context = [item['metadata']['text'] for item in top_mathces['matches']]

In [39]:
context

['profile_1.py\n\nimport ast\nimport os\nimport json\n\nclass CallGraphAnalyzer(ast.NodeVisitor):\n    def __init__(self, source):\n        self.call_graph = {}\n        self.function_code = {}\n        self.source = source\n        self.current_function = None\n\n    def visit_FunctionDef(self, node):\n        prev_function = self.current_function\n        self.current_function = node.name\n\n        # Initialize call list if not already present.\n        if node.name not in self.call_graph:\n            self.call_graph[node.name] = []\n\n        # Use ast.get_source_segment (available in Python 3.8+)\n        code_snippet = ast.get_source_segment(self.source, node)\n        self.function_code[node.name] = code_snippet if code_snippet else ""\n\n        self.generic_visit(node)\n        self.current_function = prev_function\n\n    def visit_Call(self, node):\n        if isinstance(node.func, ast.Name):\n            func_name = node.func.id\n        elif isinstance(node.func, ast.Attri

In [40]:
augmented_query = "<CONTEXT>\n" + "\n\n--------\n\n".join(context)+"\n---------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [41]:
print(augmented_query)

<CONTEXT>
profile_1.py

import ast
import os
import json

class CallGraphAnalyzer(ast.NodeVisitor):
    def __init__(self, source):
        self.call_graph = {}
        self.function_code = {}
        self.source = source
        self.current_function = None

    def visit_FunctionDef(self, node):
        prev_function = self.current_function
        self.current_function = node.name

        # Initialize call list if not already present.
        if node.name not in self.call_graph:
            self.call_graph[node.name] = []

        # Use ast.get_source_segment (available in Python 3.8+)
        code_snippet = ast.get_source_segment(self.source, node)
        self.function_code[node.name] = code_snippet if code_snippet else ""

        self.generic_visit(node)
        self.current_function = prev_function

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name):
            func_name = node.func.id
        elif isinstance(node.func, ast.Attribute):
            func

In [52]:
system_prompt = """You are a Senior Software Engineer, who is an expert in Python and fullstack development.

Answer the question I have about the codebase based on the context provided.
Always consider all of the context provided to answer my questions.
"""

llm_response = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=[
        {"role":"system", "content": system_prompt},
        {"role":"user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [53]:
#respone = perform_rag("Question")

print(response)

This codebase is a full-stack application that analyzes Python codebases to generate interactive call graphs with AI-powered explanations. Here's a breakdown of its architecture and key components:

**1. Core Analysis System**  
- **AST-based Parser** (profile_1.py & app.py):
  - Uses Python's Abstract Syntax Tree (AST) module to generate call graphs
  - Tracks function definitions and their interconnections
  - Generates structured JSON output with metadata including:
    - Function code snippets
    - Calling relationships
    - File locations
    - Directory hierarchy breadcrumbs

**2. Backend Services**  
- **Flask Web Server** (app.py):
  - REST API endpoints for data visualization and AI interactions
  - Implements code analysis and filtering (removes built-in functions)
  - Features double analysis implementation (profile_1.py/app.py needed deduplication)
  - Integration with Groq's AI service (Mixtral 8x7B model):
    - Chat endpoint for code-specific Q&A
    - Summary endpoint